## 0) Setup conda environment
Colab does not include conda by default. This cell installs Miniconda, creates the environment from `environment.yaml`, and activates it.

In [ ]:
# Install Miniconda (Colab)
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -b -p /kaggle/working/miniconda

# Initialize conda in this notebook
import os
os.environ["PATH"] = "/kaggle/working/miniconda/bin:" + os.environ["PATH"]
# Create and activate env
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r


# MyControlNet Colab Workflow
Follow the steps below to prepare data, add control weights, and train, based on the README.

## 1) Clone repo and set working directory
If the repo is already in your Drive, skip the clone and just `cd` into it.

In [ ]:
# Optional: mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Clone repo (or skip if already present in Drive)
!git clone https://github.com/robinrj6/MyControlNet.git /MyControlNet

# # Set working directory
import os
os.chdir('/MyControlNet')
print('CWD:', os.getcwd())

# !conda create -y -n control python=3.8.5
# # setup conda environment
!conda env create -f environment.yaml
!conda activate control
!python --version
# pip install ipykernel

# Link the conda env to Jupyter
# !conda run -n control python -m ipykernel install --user --name=control --display-name "Python (control)"

## 2) Download and extract Fill50K dataset
Download the dataset and extract it into the `training/` folder so you have `training/fill50k/`.

In [ ]:
# Replace with the actual dataset URL
DATASET_URL = "https://huggingface.co/lllyasviel/ControlNet/resolve/main/training/fill50k.zip"
!wget -O /content/fill50k.zip "$DATASET_URL"
!mkdir -p /content/MyControlNet/training
!unzip -q /content/fill50k.zip -d /content/MyControlNet/training

# Check folder structure
!ls -la /content/MyControlNet/training
!ls -la /content/MyControlNet/training/fill50k

## 3) Verify dataset
Run the dataset test and confirm the printed shapes look correct.

In [ ]:
# Run dataset test
!conda run -n control python tutorial_dataset_test.py 

50000
burly wood circle with orange background
(512, 512, 3)
(512, 512, 3)



In [ ]:
# Download or upload checkpoints into models/
# Example placeholders (replace with real URLs or upload manually):
# !mkdir -p models
!wget -O models/v1-5-pruned.ckpt "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt"
# !wget -O models/v2-1_512-ema-pruned.ckpt "https://huggingface.co/Manojb/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt"

# Create control checkpoint (run with conda environment)
!conda run -n control python tool_add_control.py ./models/v1-5-pruned.ckpt ./models/control_sd15_ini.ckpt


logging improved.
Enabled sliced_attention.
No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/cldm_v15.yaml]
These weights are newly added: logvar
These weights are newly added: control_model.zero_convs.0.0.weight
These weights are newly added: control_model.zero_convs.0.0.bias
These weights are newly added: control_model.zero_convs.1.0.weight
These weights are newly added: control_model.zero_convs.1.0.bias
These weights are newly added: control_model.zero_convs.2.0.weight
These weights are newly added: control_model.zero_convs.2.0.bias
These weights are newly added: control_model.zero_convs.3.0.weight
These weights are newly added: control_model.zero_convs.3.0.bias
These weights are newly added: control_model

## 5) Train
Start training after dataset and checkpoints are ready.

In [ ]:
# Start training
# !python tutorial_train.py